# Performance analysis

In [ ]:
%matplotlib inline
import re
from datetime import datetime
import numpy as np
from mpmath import mp
import matplotlib as mpl
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import host_subplot
import mpl_toolkits.axisartist as AA
import math
import warnings
from ipywidgets import interact, interactive, fixed, interact_manual
plt.rcParams['figure.dpi'] = 90
plt.rcParams['figure.figsize'] = [24.0, 16.0]
plt.rcParams['text.latex.unicode'] = False
plt.rcParams['text.usetex'] = False
plt.rcParams['mathtext.fontset'] = 'stixsans'
plt.rcParams['font.family'] = 'DejaVu Sans'

## Log parsing

In [ ]:
def events(log):
    start = None
    events = []
    for line in log.split('\n'):
        match = re.search(r'^\[(.+Z) TRACE [:\w]+\] (BEGIN|END) (.*)$', line)
        if match:
            (time, event, name) = match.groups()
            time = datetime.strptime(time, '%Y-%m-%dT%H:%M:%S.%fZ')
            if start:
                time -= start
                time = time.total_seconds()
            else:
                start = time
                time = 0
            begin = event == 'BEGIN'
            events += [(time, begin, name)]
    return events

In [ ]:
def process(events):
    stack = []
    spans = []
    histogram = {}
    last = None
    for event in events:
        (time, begin, name) = event
        # Collect top of stack histogram
        if last:
            top_of_stack = stack[-1][2]
            duration = time - last
            if top_of_stack in histogram:
                histogram[top_of_stack] += duration
            else:
                histogram[top_of_stack] = duration
        last = time
        # Compute spans
        if begin:
            stack += [event]
        else:
            (start, _, previous_name) = stack[-1]
            stack = stack[:-1]
            assert name == previous_name
            spans += [(len(stack), start, time, name)]
    histogram = {k: v for k, v in sorted(histogram.items(), key=lambda item: -item[1])}
    print(len(spans), len(histogram))
    return spans, histogram

In [ ]:
x = []
data = []
for i in range(10, 16):
    i = 2**i
    x += [i]
    data += [process(events(open("../../pedersen-" + str(i) + ".log","r").read()))]
print(x)

## Flame graph

In [ ]:
def flamegraph(spans):
    labels = list(set([span[3] for span in spans]))
    maxt = spans[-1][2]
    colour_map = mpl.cm.rainbow(np.linspace(0, 1, len(labels)))
    fig = plt.figure()
    ax = fig.add_subplot(111)
    ax.set_xlim((0,maxt))
    ax.set_ylim((0,7))

    for span in spans:
        (depth, start, end, label) = span
        colour = colour_map[labels.index(label)]
        rectangle = mpl.patches.Rectangle((start, depth), end - start, 1, fc=colour, ec='white')
        ax.add_patch(rectangle)
        if end - start > 0.01 * maxt:
            ax.text(start + 0.005 * maxt, depth+0.05, label)

In [ ]:
flamegraph(process(events(open("../../pedersen-test.log","r").read()))[0])

In [ ]:
flamegraph(process(events(open("../../pedersen-test3.log","r").read()))[0])

In [ ]:
flamegraph(data[-1][0])

## Histogram

In [ ]:
labels = list(data[-1][1].keys())
values = np.array([[hist[label] for label in labels] for (_, hist) in data]).T
fig, ax = plt.subplots()
y = values / np.sum(values, axis=0)
ax.stackplot(np.log2(x), y[::-1, :], labels=labels[::-1])
ax.set_xlim((10,15))
ax.set_ylim((0,1))

handles, labels = ax.get_legend_handles_labels()
ax.legend(handles[::-1], labels[::-1], loc='lower left')

plt.show()

In [ ]:
values 

In [ ]:
2**13